In [1]:
import pandas as pd; pd.set_option('display.max_columns', 100)
import numpy as np

from tqdm.notebook import tqdm

import re
import os

from functools import partial
from scipy.stats import mode

import matplotlib.pyplot as plt; plt.style.use('ggplot')
import seaborn as sns
import plotly.express as px

from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.preprocessing import MinMaxScaler, StandardScaler, LabelEncoder, FunctionTransformer
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.compose import ColumnTransformer
from sklearn.impute import KNNImputer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.model_selection import KFold, StratifiedKFold, train_test_split, GridSearchCV, RepeatedStratifiedKFold
from sklearn.metrics import roc_auc_score, roc_curve, RocCurveDisplay, cohen_kappa_score, log_loss, f1_score
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_selection import RFE, RFECV
from sklearn.isotonic import IsotonicRegression
from sklearn.calibration import CalibrationDisplay
from sklearn.inspection import PartialDependenceDisplay, permutation_importance
from sklearn.linear_model import LogisticRegression
from collections import Counter
from sklearn.ensemble import RandomForestClassifier, HistGradientBoostingClassifier, GradientBoostingClassifier, ExtraTreesClassifier
from sklearn.svm import SVC
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier

from sklearn.manifold import TSNE
# import optuna

from View import MyClass

In [16]:
eval_metric = False
params_bc = {
                    "n_estimators": 2000,
                    "learning_rate": 0.001,
                    "loss_function": "Logloss",
                    "eval_metric": input("введите метрику: ") if eval_metric else "AUC",
                    "task_type": "CPU",
                    "max_bin": 20,
                    "verbose": False,
                    "max_depth": 6,
                    "l2_leaf_reg": 10,
                    "early_stopping_rounds": 50,
                    "thread_count": 6,
                    "random_seed": 42
                }
params_bc

{'n_estimators': 2000,
 'learning_rate': 0.001,
 'loss_function': 'Logloss',
 'eval_metric': 'AUC',
 'task_type': 'CPU',
 'max_bin': 20,
 'verbose': False,
 'max_depth': 6,
 'l2_leaf_reg': 10,
 'early_stopping_rounds': 50,
 'thread_count': 6,
 'random_seed': 42}

In [98]:
mc = MyClass()

In [99]:
print(mc.__doc__)

Визуализация и предобработка данных
# Первичный обзор
* Метод "view_csv" принимает DataFrame и выводит данные о нем, переводит название столбцов в lower
возвращает data.columns, categorical, numerical

# Статистика
* Метод "stat_frame" принимает DataFrame и возвращает DataFrame со статистиками
* Метод "corr" принимает DataFrame и target : str
* Метод corr_sign принимает data: DataFrame, drop_sign:List ( корреляция между признаками)
* Метод nonlianer_comun принимает data: DataFrame, numerical: list(список числовых признаков
target: str(по умолчанию "class"), num_sign: int(кол-во столбцов с нелинейной связью)
* Метод "percentile_99_1" принимает data: DataFrame, data_stat : DataFrame( возвращает метод "stat_frame"), flag: bool(
если флаг равен True возвращает фрейм где максимум заменен на 99 перцентиль, а минимум на 1 перцентиль)
по умолчанию flag = False
возвращат данные где больше 99 перцетиля и меньше 1(используется метод for, не рекомендуется где очень много столбцов)

# Работа с данн

<a id="3"></a>
# <h1 style="background-color:lightgray;font-family:newtimeroman;font-size:350%;text-align:center;border-radius: 15px 50px;">Reading Data Files</h1> 

In [100]:
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')
submission = pd.read_csv('data/sample_submission.csv')

print('The dimension of the train dataset is:', train.shape)
print('The dimension of the test dataset is:', test.shape)

The dimension of the train dataset is: (101763, 23)
The dimension of the test dataset is: (67842, 22)


In [101]:
train.describe()

,id,loc,v(g),ev(g),iv(g),n,v,l,d,i,e,b,t,lOCode,lOComment,lOBlank,locCodeAndComment,uniq_Op,uniq_Opnd,total_Op,total_Opnd,branchCount
count,101763.000000,101763.000000,101763.000000,101763.000000,101763.000000,101763.000000,101763.000000,101763.000000,101763.000000,101763.000000,1.017630e+05,101763.000000,101763.000000,101763.000000,101763.000000,101763.000000,101763.000000,101763.000000,101763.000000,101763.000000,101763.000000,101763.000000
mean,50881.000000,37.347160,5.492684,2.845022,3.498826,96.655995,538.280956,0.111634,13.681881,27.573007,2.085359e+04,0.179164,1141.357982,22.802453,1.773945,3.979865,0.196604,11.896131,15.596671,57.628116,39.249698,9.839549
std,29376.592059,54.600401,7.900855,4.631262,5.534541,171.147191,1270.791601,0.100096,14.121306,22.856742,1.905714e+05,0.421844,9862.795472,38.541010,5.902412,6.382358,0.998906,6.749549,18.064261,104.537660,71.692309,14.412769
min,0.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
25%,25440.500000,13.000000,2.000000,1.000000,1.000000,25.000000,97.670000,0.050000,5.600000,15.560000,5.647300e+02,0.030000,31.380000,7.000000,0.000000,1.000000,0.000000,8.000000,7.000000,15.000000,10.000000,3.000000
50%,50881.000000,22.000000,3.000000,1.000000,2.000000,51.000000,232.790000,0.090000,9.820000,23.360000,2.256230e+03,0.080000,125.400000,14.000000,0.000000,2.000000,0.000000,11.000000,12.000000,30.000000,20.000000,5.000000
75%,76321.500000,42.000000,6.000000,3.000000,4.000000,111.000000,560.250000,0.150000,18.000000,34.340000,1.019324e+04,0.190000,565.920000,26.000000,1.000000,5.000000,0.000000,16.000000,20.000000,66.000000,45.000000,11.000000
max,101762.000000,3442.000000,404.000000,165.000000,402.000000,8441.000000,80843.080000,1.000000,418.200000,569.780000,1.684662e+07,26.950000,935923.390000,2824.000000,344.000000,219.000000,43.000000,410.000000,1026.000000,5420.000000,3021.000000,503.000000


In [102]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 101763 entries, 0 to 101762
Data columns (total 23 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   id                 101763 non-null  int64  
 1   loc                101763 non-null  float64
 2   v(g)               101763 non-null  float64
 3   ev(g)              101763 non-null  float64
 4   iv(g)              101763 non-null  float64
 5   n                  101763 non-null  float64
 6   v                  101763 non-null  float64
 7   l                  101763 non-null  float64
 8   d                  101763 non-null  float64
 9   i                  101763 non-null  float64
 10  e                  101763 non-null  float64
 11  b                  101763 non-null  float64
 12  t                  101763 non-null  float64
 13  lOCode             101763 non-null  int64  
 14  lOComment          101763 non-null  int64  
 15  lOBlank            101763 non-null  int64  
 16  lo

In [103]:
test.describe()

,id,loc,v(g),ev(g),iv(g),n,v,l,d,i,e,b,t,lOCode,lOComment,lOBlank,locCodeAndComment,uniq_Op,uniq_Opnd,total_Op,total_Opnd,branchCount
count,67842.000000,67842.000000,67842.000000,67842.000000,67842.000000,67842.000000,67842.000000,67842.000000,67842.000000,67842.000000,6.784200e+04,67842.000000,67842.000000,67842.000000,67842.000000,67842.000000,67842.000000,67842.000000,67842.000000,67842.000000,67842.000000,67842.000000
mean,135683.500000,37.855945,5.559462,2.857699,3.535715,98.222178,548.422497,0.111521,13.762472,27.635377,2.216432e+04,0.182075,1218.682009,23.131924,1.832493,4.020739,0.200834,11.909776,15.678370,58.325212,39.845258,9.985923
std,19584.442818,55.793918,8.237389,4.666370,5.339085,179.123492,1318.416828,0.100840,14.487425,23.420521,2.072770e+05,0.418493,10739.081373,39.606659,6.229052,7.136069,1.044581,6.974254,17.979368,105.491447,72.756320,15.342975
min,101763.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
25%,118723.250000,13.000000,2.000000,1.000000,1.000000,25.000000,96.210000,0.040000,5.500000,15.540000,5.629400e+02,0.030000,31.270000,7.000000,0.000000,1.000000,0.000000,8.000000,7.000000,15.000000,10.000000,3.000000
50%,135683.500000,22.000000,3.000000,1.000000,2.000000,51.000000,232.790000,0.090000,9.820000,23.230000,2.276780e+03,0.080000,126.490000,14.000000,0.000000,2.000000,0.000000,12.000000,12.000000,30.000000,20.000000,5.000000
75%,152643.750000,42.000000,6.000000,3.000000,4.000000,111.000000,560.250000,0.140000,18.000000,34.270000,1.011243e+04,0.190000,561.802500,26.000000,1.000000,5.000000,0.000000,16.000000,20.000000,66.000000,44.000000,11.000000
max,169604.000000,1891.000000,470.000000,165.000000,402.000000,8441.000000,80843.080000,1.000000,418.200000,569.780000,1.684662e+07,18.380000,935923.390000,1599.000000,344.000000,447.000000,42.000000,411.000000,1026.000000,3368.000000,2301.000000,826.000000


Ни в обучающих, ни в наборах тестовых данных нет пропущенных значений. Поскольку это синтетический набор данных, в качестве проверки на работоспособность мы проверим наличие дубликатов. Сначала давайте взглянем на набор данных о поездах.

In [104]:
print(f"Если мы удалим id из набора данных обучения, то измерение будет {train.drop(columns = ['id'], axis = 1).drop_duplicates().shape}")
print(f"Если мы удалим id и defects из набора данных обучения, то измерение будет {train.drop(columns = ['id', 'defects'], axis = 1).drop_duplicates().shape}")
print(f"Если мы удалим id, branchCount и defects из набора данных обучения, то измерение будет {train.drop(columns = ['id', 'branchCount', 'defects'], axis = 1).drop_duplicates().shape}")

Если мы удалим id из набора данных обучения, то измерение будет (101763, 22)
Если мы удалим id и defects из набора данных обучения, то измерение будет (101763, 21)
Если мы удалим id, branchCount и defects из набора данных обучения, то измерение будет (101685, 20)


Из вышеизложенного мы видим, что в наборе данных обучения имеется 78 дублированых наблюдений. Теперь давайте взглянем на тестовый набор данных.

In [105]:
print(f"Если мы удалим id из набора данных test, то измерение будет {test.drop(columns = ['id'], axis = 1).drop_duplicates().shape}")
print(f"Если мы удалим id, branchCount из набора данных test, то измерение будет {test.drop(columns = ['id', 'branchCount'], axis = 1).drop_duplicates().shape}")

Если мы удалим id из набора данных test, то измерение будет (67842, 21)
Если мы удалим id, branchCount из набора данных test, то измерение будет (67804, 20)


Из вышеизложенного мы видим, что в тестовом наборе данных имеется 38 дублированых наблюдений. Наконец, давайте проверим, есть ли наблюдения, которые появляются как в наборах обучающих, так и в тестовых данных.

In [106]:
to_check = pd.merge(train.drop(columns = ['id', 'defects'], axis = 1), test.drop(columns = 'id', axis = 1))
print(f"Есть {to_check.shape[0]} дублированные наблюдения в наборах обучающих и тестовых данных")

to_check = pd.merge(train.drop(columns = ['id', 'branchCount', 'defects'], axis = 1), test.drop(columns = ['id', 'branchCount'], axis = 1), on = test.drop(columns = ['id', 'branchCount'], axis = 1).columns.tolist())
print(f"Есть {to_check.drop_duplicates().shape[0]} дублированные наблюдения в наборах обучающих и тестовых данных")

Есть 0 дублированные наблюдения в наборах обучающих и тестовых данных
Есть 116 дублированные наблюдения в наборах обучающих и тестовых данных


In [107]:
stat_train = mc.stat_frame(train)
stat_train

,count,max,quantile_99,max_vs_quantile_99,min,quantile_1,min_vs_quantile_1,nunique,isna_sum,dtype
id,101763,101762,100744.38,1018,0,1017.62,1018,101763,0,int64
loc,101763,3442.0,226.0,1011,1.0,4.0,24,378,0,float64
v(g),101763,404.0,33.38,1018,1.0,1.0,0,106,0,float64
ev(g),101763,165.0,23.0,855,1.0,1.0,0,71,0,float64
iv(g),101763,402.0,19.0,963,1.0,1.0,0,84,0,float64
n,101763,8441.0,756.0,1017,0.0,0.0,0,836,0,float64
v,101763,80843.08,5033.79,1014,0.0,0.0,0,4515,0,float64
l,101763,1.0,0.5,360,0.0,0.0,0,55,0,float64
d,101763,418.2,62.37,1017,0.0,0.0,0,3360,0,float64
i,101763,569.78,107.69,1012,0.0,0.0,0,5171,0,float64


In [108]:
train['defects'] = train['defects'].astype(int)

In [109]:
# замена на 99 перцентиля на макс, 1 min
data_perc = train.copy()
for col in data_perc.columns:
    len_more_quant = data_perc.loc[data_perc[col] > data_perc[col].quantile(0.99)].shape[0]
    len_less_quant = data_perc.loc[data_perc[col] < data_perc[col].quantile(0.01)].shape[0]
    if len_more_quant > 0:
        data_perc.loc[data_perc[col] > data_perc[col].quantile(0.99), col] = data_perc[col].quantile(0.99)
    if len_less_quant > 0:
        data_perc.loc[data_perc[col] < data_perc[col].quantile(0.01), col] = data_perc[col].quantile(0.01)

In [110]:
stat_train = mc.stat_frame(data_perc)
stat_train

,count,max,quantile_99,max_vs_quantile_99,min,quantile_1,min_vs_quantile_1,nunique,isna_sum,dtype
id,101763,100744.38,100744.1444,1018,1017.62,1017.8556,1018,99729,0,float64
loc,101763,226.0,226.0,0,4.0,4.0,0,219,0,float64
v(g),101763,33.38,33.1444,1018,1.0,1.0,0,34,0,float64
ev(g),101763,23.0,23.0,0,1.0,1.0,0,23,0,float64
iv(g),101763,19.0,19.0,0,1.0,1.0,0,19,0,float64
n,101763,756.0,756.0,0,0.0,0.0,0,630,0,float64
v,101763,5033.79,5033.79,0,0.0,0.0,0,4278,0,float64
l,101763,0.5,0.5,0,0.0,0.0,0,50,0,float64
d,101763,62.37,62.37,0,0.0,0.0,0,3112,0,float64
i,101763,107.69,107.69,0,0.0,0.0,0,4885,0,float64


In [111]:
train = data_perc

In [112]:
train['loc'] = np.log10(train['loc'])
test['loc'] = np.log10(test['loc'])

In [113]:
train = train.drop(['id', 'branchCount'], axis=1)
test = test.drop(['id', 'branchCount'], axis=1)

<a id="5"></a>
# <h1 style="background-color:lightgray;font-family:newtimeroman;font-size:350%;text-align:center;border-radius: 15px 50px;">Baseline Modeling 1.0</h1>

Во-первых, мы начинаем с создания некоторых стандартных моделей без разработки функциональных возможностей или HPO. Сначала мы определяем входные и целевые функции.

In [114]:
# X = train.drop(columns = ['defects','v', 'n', 'e', 'b', 'total_Opnd'], axis = 1)
X = train.drop(columns = ['defects'], axis = 1)
Y = train['defects']

# test_cv = test.drop(columns = ['v', 'n', 'e', 'b', 'total_Opnd'], axis = 1)
test_cv = test

Сначала мы определяем функцию ансамбля Хилла для прогнозирования модели ансамбля.

In [115]:
def hill_climbing(x, y, x_test):
    
    # Evaluating oof predictions
    scores = {}
    for col in x.columns:
        scores[col] = roc_auc_score(y, x[col])

    # Sorting the model scores
    scores = {k: v for k, v in sorted(scores.items(), key = lambda item: item[1], reverse = True)}

    # Sort oof_df and test_preds
    x = x[list(scores.keys())]
    x_test = x_test[list(scores.keys())]

    STOP = False
    current_best_ensemble = x.iloc[:,0]
    current_best_test_preds = x_test.iloc[:,0]
    MODELS = x.iloc[:,1:]
    weight_range = np.arange(-0.5, 0.51, 0.01) 
    history = [roc_auc_score(y, current_best_ensemble)]
    j = 0

    while not STOP:
        j += 1
        potential_new_best_cv_score = roc_auc_score(y, current_best_ensemble)
        k_best, wgt_best = None, None
        for k in MODELS:
            for wgt in weight_range:
                potential_ensemble = (1 - wgt) * current_best_ensemble + wgt * MODELS[k]
                cv_score = roc_auc_score(y, potential_ensemble)
                if cv_score > potential_new_best_cv_score:
                    potential_new_best_cv_score = cv_score
                    k_best, wgt_best = k, wgt

        if k_best is not None:
            current_best_ensemble = (1 - wgt_best) * current_best_ensemble + wgt_best * MODELS[k_best]
            current_best_test_preds = (1 - wgt_best) * current_best_test_preds + wgt_best * x_test[k_best]
            MODELS.drop(k_best, axis = 1, inplace = True)
            if MODELS.shape[1] == 0:
                STOP = True
            history.append(potential_new_best_cv_score)
        else:
            STOP = True
        
    hill_ens_pred_1 = current_best_ensemble
    hill_ens_pred_2 = current_best_test_preds
    
    return [hill_ens_pred_1, hill_ens_pred_2]

Затем мы создаем несколько стандартных моделей в рамках процедуры 10-кратной перекрестной проверки.

In [116]:


params_cb = {
    "n_estimators": 700,
    'subsample': 0.3,
    'max_depth': 9,
    'leaf_estimation_iterations': 50,
    'l2_leaf_reg': 30,
    "learning_rate": 0.01,
    'colsample_bylevel': 0.3,
    "loss_function": "Logloss",
    "eval_metric": "AUC",
    "task_type": "CPU",
    "max_bin": 20,
    "verbose": False,
    "early_stopping_rounds": 50,
    "thread_count": 6,
    "random_seed": 42,
}

params_xgb = {
    "n_estimators" : 700,
    "gamma" : 2,
    "subsample" : 0.3,
    "min_child_weight" : 10,
    "booster": "gbtree",
    "eval_metric": "auc",
    "objective": "binary:logistic",
    "learning_rate": 0.01,
    "nthread": 6,
    "max_depth" : 9,
    "seed": 27
    }

params_lgb = {
    "n_estimators" : 700,
    "objective" : 'binary',
    "subsample" : 0.3,
    "num_leaves" : 20,
    "reg_alpha": 3,
    "reg_lambda": 3,
    "learning_rate": 0.01,
    "max_depth" : 9,
    "seed": 27,
    "colsample_bytree": 0.7
    }


params_rf = {
    "max_leaf_nodes" : 92,
    "n_estimators" : 700,
    "max_depth" : 9,
    "min_samples_split": 15,
    "min_samples_leaf": 10,
    }

params_et = {
    "max_leaf_nodes" : 92,
    "n_estimators" : 700,
    "max_depth" : 9,
    "min_samples_split": 15,
    "min_samples_leaf": 10,
    }

params_gb = {
    "l2_regularization": 0.01,
    "learning_rate" : 0.01,
    "max_iter" : 700,
    "max_bins" : 255,
    "max_depth" : 9,
    "min_samples_leaf": 15,
    "max_leaf_nodes": 92,
    }



In [117]:
ens_cv_scores, ens_preds = list(), list()
hill_ens_cv_scores, hill_ens_preds =  list(), list()

sk = RepeatedStratifiedKFold(n_splits = 5, n_repeats = 1, random_state = 42)
for i, (train_idx, test_idx) in enumerate(sk.split(X, Y)):

    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    Y_train, Y_test = Y.iloc[train_idx], Y.iloc[test_idx]
    
    print('----------------------------------------------------------')
    
    ########
    ## RF ##
    ########

    RF_md = RandomForestClassifier(n_estimators = 500, 
                                   max_depth = 7,
                                   min_samples_split = 15,
                                   min_samples_leaf = 10).fit(X_train, Y_train)
    
    RF_pred = RF_md.predict_proba(X_test)[:, 1]
    RF_score = roc_auc_score(Y_test, RF_pred)

    print('Fold', i, '==> RF oof ROC-AUC score is ==>', RF_score)

    RF_pred_test = RF_md.predict_proba(test_cv)[:, 1]
    
    #################
    ## Extra Trees ##
    #################

    ET_md = ExtraTreesClassifier(n_estimators = 500, 
                                 max_depth = 7,
                                 min_samples_split = 15,
                                 min_samples_leaf = 10).fit(X_train, Y_train)

    ET_pred = ET_md.predict_proba(X_test)[:, 1]
    ET_score = roc_auc_score(Y_test, ET_pred)

    print('Fold', i, '==> ET oof ROC-AUC score is ==>', ET_score)

    ET_pred_test = ET_md.predict_proba(test_cv)[:, 1]

    ##########################
    ## HistGradientBoosting ##
    ##########################

    hist_md = HistGradientBoostingClassifier(l2_regularization = 0.01,
                                             early_stopping = False,
                                             learning_rate = 0.01,
                                             max_iter = 500,
                                             max_depth = 5,
                                             max_bins = 255,
                                             min_samples_leaf = 15,
                                             max_leaf_nodes = 10).fit(X_train, Y_train)
    
    hist_pred = hist_md.predict_proba(X_test)[:, 1]
    hist_score = roc_auc_score(Y_test, hist_pred)

    print('Fold', i, '==> Hist oof ROC-AUC score is ==>', hist_score)  

    hist_pred_test = hist_md.predict_proba(test_cv)[:, 1]

    ##########
    ## LGBM ##
    ##########

    LGBM_md = LGBMClassifier(objective = 'binary',
                             n_estimators = 500,
                             max_depth = 7,
                             learning_rate = 0.01,
                             num_leaves = 20,
                             reg_alpha = 3,
                             reg_lambda = 3,
                             subsample = 0.7,
                             colsample_bytree = 0.7).fit(X_train, Y_train)

    lgb_pred = LGBM_md.predict_proba(X_test)[:, 1]
    lgb_score = roc_auc_score(Y_test, lgb_pred)

    print('Fold', i, '==> LGBM oof ROC-AUC score is ==>', lgb_score) 

    lgb_pred_test = LGBM_md.predict_proba(test_cv)[:, 1]

    #########
    ## XGB ##
    #########

    XGB_md = XGBClassifier(objective = 'binary:logistic',
                           tree_method = 'hist',
                           colsample_bytree = 0.7, 
                           gamma = 2, 
                           learning_rate = 0.01, 
                           max_depth = 7, 
                           min_child_weight = 10, 
                           n_estimators = 500, 
                           subsample = 0.7).fit(X_train, Y_train)

    xgb_pred = XGB_md.predict_proba(X_test)[:, 1]
    xgb_score = roc_auc_score(Y_test, xgb_pred)

    print('Fold', i, '==> XGB oof ROC-AUC score is ==>', xgb_score)

    xgb_pred_test = XGB_md.predict_proba(test_cv)[:, 1]

    ##############
    ## CatBoost ##
    ##############

    Cat_md = CatBoostClassifier(loss_function = 'Logloss',
                                iterations = 500,
                                learning_rate = 0.01,
                                depth = 7,
                                random_strength = 0.5,
                                bagging_temperature = 0.7,
                                border_count = 30,
                                l2_leaf_reg = 5,
                                verbose = False, 
                                task_type = 'CPU').fit(X_train, Y_train)

    cat_pred = Cat_md.predict_proba(X_test)[:, 1]
    cat_score = roc_auc_score(Y_test, cat_pred)

    print('Fold', i, '==> CatBoost oof ROC-AUC score is ==>', cat_score)

    cat_pred_test = Cat_md.predict_proba(test_cv)[:, 1]    
    
    ##############
    ## Ensemble ##
    ##############
    
    ens_pred_1 = (RF_pred + ET_pred + hist_pred + lgb_pred + xgb_pred + cat_pred) / 6
    ens_pred_2 = (RF_pred_test + ET_pred_test + hist_pred_test + lgb_pred_test + xgb_pred_test + cat_pred_test) / 6
    
    ens_score_fold = roc_auc_score(Y_test, ens_pred_1)
    ens_cv_scores.append(ens_score_fold)
    ens_preds.append(ens_pred_2)
    
    print('Fold', i, '==> Average Ensemble oof ROC-AUC score is ==>', ens_score_fold)
    
    ############################
    ## Hill Climbing Ensemble ##
    ############################
    
    x = pd.DataFrame({'RF': RF_pred,
                      'ET': ET_pred, 
                      'Hist': hist_pred, 
                      'LGBM': lgb_pred,
                      'XGB': xgb_pred,
                      'Cat': cat_pred})
    y = Y_test
        
    x_test = pd.DataFrame({'RF': RF_pred_test,
                           'ET': ET_pred_test, 
                           'Hist': hist_pred_test, 
                           'LGBM': lgb_pred_test,
                           'XGB': xgb_pred_test,
                           'Cat': cat_pred_test})
    
    hill_results = hill_climbing(x, y, x_test)
    
    hill_ens_score_fold = roc_auc_score(y, hill_results[0])
    hill_ens_cv_scores.append(hill_ens_score_fold)
    hill_ens_preds.append(hill_results[1])

    print('Fold', i, '==> Hill Climbing Ensemble oof ROC-AUC score is ==>', hill_ens_score_fold)

In [118]:
# ens_cv_scores, ens_preds, rf_scores, et_scores, hist_scores, lgbm_scores, xgb_scores, cb_scores = list(), list(), list(), list(), list(), list(), list(), list() 
# hill_ens_cv_scores, hill_ens_preds =  list(), list()

# sk = RepeatedStratifiedKFold(n_splits = 5, n_repeats = 1, random_state = 42)
# for i, (train_idx, test_idx) in enumerate(sk.split(X, Y)):

#     X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
#     Y_train, Y_test = Y.iloc[train_idx], Y.iloc[test_idx]
    
#     print('----------------------------------------------------------')
    
#     ########
#     ## RF ##
#     ########

#     RF_md = RandomForestClassifier(**params_rf).fit(X_train, Y_train)

    
#     RF_pred = RF_md.predict_proba(X_test)[:, 1]
# #     RF_pred = RF_md.predict(X_test)
#     RF_score = roc_auc_score(Y_test, RF_pred)
#     rf_scores.append(RF_score)

#     print('Fold', i, '==> RF oof ROC-AUC score is ==>', RF_score)

#     RF_pred_test = RF_md.predict_proba(test_cv)[:, 1]
# #     RF_pred_test = RF_md.predict(test_cv)
    
#     #################
#     ## Extra Trees ##
#     #################

#     ET_md = ExtraTreesClassifier(**params_et).fit(X_train, Y_train)


#     ET_pred = ET_md.predict_proba(X_test)[:, 1]
# #     ET_pred = ET_md.predict(X_test)
#     ET_score = roc_auc_score(Y_test, ET_pred)
#     et_scores.append(RF_score)

#     print('Fold', i, '==> ET oof ROC-AUC score is ==>', ET_score)

#     ET_pred_test = ET_md.predict_proba(test_cv)[:, 1]
# #     ET_pred_test = ET_md.predict(test_cv)

#     ##########################
#     ## HistGradientBoosting ##
#     ##########################

#     hist_md = HistGradientBoostingClassifier(**params_gb).fit(X_train, Y_train)

    
#     hist_pred = hist_md.predict_proba(X_test)[:, 1]
# #     hist_pred = hist_md.predict(X_test)
#     hist_score = roc_auc_score(Y_test, hist_pred)
#     hist_scores.append(RF_score)

#     print('Fold', i, '==> Hist oof ROC-AUC score is ==>', hist_score)  

#     hist_pred_test = hist_md.predict_proba(test_cv)[:, 1]
# #     hist_pred_test = hist_md.predict(test_cv) 

#     ##########
#     ## LGBM ##
#     ##########

#     LGBM_md = LGBMClassifier(**params_lgb).fit(X_train, Y_train)


#     lgb_pred = LGBM_md.predict_proba(X_test)[:, 1]
# #     lgb_pred = LGBM_md.predict(X_test)
#     lgb_score = roc_auc_score(Y_test, lgb_pred)
#     lgbm_scores.append(RF_score)

#     print('Fold', i, '==> LGBM oof ROC-AUC score is ==>', lgb_score) 

#     lgb_pred_test = LGBM_md.predict_proba(test_cv)[:, 1]
# #     lgb_pred_test = LGBM_md.predict(test_cv) 

#     #########
#     ## XGB ##
#     #########

#     XGB_md = XGBClassifier(**params_xgb).fit(X_train, Y_train)


#     xgb_pred = XGB_md.predict_proba(X_test)[:, 1]
# #     xgb_pred = XGB_md.predict(X_test)
#     xgb_score = roc_auc_score(Y_test, xgb_pred)
#     xgb_scores.append(RF_score)

#     print('Fold', i, '==> XGB oof ROC-AUC score is ==>', xgb_score)

#     xgb_pred_test = XGB_md.predict_proba(test_cv)[:, 1]
# #     xgb_pred_test = XGB_md.predict(test_cv) 

#     ##############
#     ## CatBoost ##
#     ##############

#     Cat_md = CatBoostClassifier(**params_cb).fit(X_train, Y_train)


#     cat_pred = Cat_md.predict_proba(X_test)[:, 1]
# #     cat_pred = Cat_md.predict(X_test)
#     cat_score = roc_auc_score(Y_test, cat_pred)
#     cb_scores.append(RF_score)

#     print('Fold', i, '==> CatBoost oof ROC-AUC score is ==>', cat_score)

#     cat_pred_test = Cat_md.predict_proba(test_cv)[:, 1]    
# #     cat_pred_test = Cat_md.predict(test_cv)  
    
#     ##############
#     ## Ensemble ##
#     ##############
    
#     ens_pred_1 = (RF_pred + ET_pred + hist_pred + lgb_pred + xgb_pred + cat_pred) / 6
#     ens_pred_2 = (RF_pred_test + ET_pred_test + hist_pred_test + lgb_pred_test + xgb_pred_test + cat_pred_test) / 6
    
#     ens_score_fold = roc_auc_score(Y_test, ens_pred_1)
#     ens_cv_scores.append(ens_score_fold)
#     ens_preds.append(ens_pred_2)
    
#     print('Fold', i, '==> Average Ensemble oof ROC-AUC score is ==>', ens_score_fold)
    
#     ############################
#     ## Hill Climbing Ensemble ##
#     ############################
    
#     x = pd.DataFrame({'RF': RF_pred,
#                       'ET': ET_pred, 
#                       'Hist': hist_pred, 
#                       'LGBM': lgb_pred,
#                       'XGB': xgb_pred,
#                       'Cat': cat_pred})
#     y = Y_test
        
#     x_test = pd.DataFrame({'RF': RF_pred_test,
#                            'ET': ET_pred_test, 
#                            'Hist': hist_pred_test, 
#                            'LGBM': lgb_pred_test,
#                            'XGB': xgb_pred_test,
#                            'Cat': cat_pred_test})
    
#     hill_results = hill_climbing(x, y, x_test)
    
#     hill_ens_score_fold = roc_auc_score(y, hill_results[0])
#     hill_ens_cv_scores.append(hill_ens_score_fold)
#     hill_ens_preds.append(hill_results[1])

#     print('Fold', i, '==> Hill Climbing Ensemble oof ROC-AUC score is ==>', hill_ens_score_fold)

----------------------------------------------------------
Fold 0 ==> RF oof ROC-AUC score is ==> 0.7916226200139873
Fold 0 ==> ET oof ROC-AUC score is ==> 0.786455196366492
Fold 0 ==> Hist oof ROC-AUC score is ==> 0.7950583484440277
Fold 0 ==> LGBM oof ROC-AUC score is ==> 0.7960781929197938
Fold 0 ==> XGB oof ROC-AUC score is ==> 0.7952200441214832
Fold 0 ==> CatBoost oof ROC-AUC score is ==> 0.7944885262934347
Fold 0 ==> Average Ensemble oof ROC-AUC score is ==> 0.7951980908051964
Fold 0 ==> Hill Climbing Ensemble oof ROC-AUC score is ==> 0.7963780608969018
----------------------------------------------------------
Fold 1 ==> RF oof ROC-AUC score is ==> 0.7895056330777255
Fold 1 ==> ET oof ROC-AUC score is ==> 0.7838119220555355
Fold 1 ==> Hist oof ROC-AUC score is ==> 0.7927747352862512
Fold 1 ==> LGBM oof ROC-AUC score is ==> 0.794089441446484
Fold 1 ==> XGB oof ROC-AUC score is ==> 0.7937673860211087
Fold 1 ==> CatBoost oof ROC-AUC score is ==> 0.7920708174869595
Fold 1 ==> Avera

In [119]:
print('Средняя совокупность оценок ROC-AUC за 10-кратный период составляет', np.mean(ens_cv_scores))
print('Hill Climbing Ensemble ROC-AUC набрал более 10 баллов, это', np.mean(hill_ens_cv_scores))

Средняя совокупность оценок ROC-AUC за 10-кратный период составляет 0.7919170014251794
Hill Climbing Ensemble ROC-AUC набрал более 10 баллов, это 0.7929947726015047


Средняя совокупность оценок ROC-AUC за 10-кратный период составляет 0.791766936860115  
Hill Climbing Ensemble ROC-AUC набрал более 10 баллов, это 0.7933394346455375

In [120]:
ens_preds_test = pd.DataFrame(ens_preds).apply(np.mean, axis = 0)

submission['defects'] = ens_preds_test
submission.to_csv('ens_sub.csv', index = False)

ens_preds_test = pd.DataFrame(hill_ens_preds).apply(np.mean, axis = 0)

submission['defects'] = ens_preds_test
submission.to_csv('hill_sub.csv', index = False)